# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Häberle  ->  M. Häberle  |  ['M. Häberle']
T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 69 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2406.17347


extracting tarball to tmp_2406.17347...

 done.


M. Häberle  ->  M. Häberle  |  ['M. Häberle']


Found 116 bibliographic references in tmp_2406.17347/niederhofer24.bbl.
Retrieving document from  https://arxiv.org/e-print/2406.17429
extracting tarball to tmp_2406.17429...

 done.


Found 75 bibliographic references in tmp_2406.17429/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.17347-b31b1b.svg)](https://arxiv.org/abs/2406.17347) | **Hubble Space Telescope proper motions of Large Magellanic Cloud star clusters -- I. Catalogues and results for NGC 1850**  |
|| F. Niederhofer, et al. -- incl., <mark>M. Häberle</mark> |
|*Appeared on*| *2024-06-26*|
|*Comments*| *27 pages, 15 figures. Accepted for publication in A&A. Astro-photometric catalogues are available at this https URL*|
|**Abstract**|            We present proper motion (PM) measurements for a sample of 23 massive star clusters within the Large Magellanic Cloud using multi-epoch data from the Hubble Space Telescope (HST). We combined archival data from the ACS/WFC and WFC3/UVIS instruments with observations from a dedicated HST programme, resulting in time baselines between 4.7 and 18.2 yr available for PM determinations. For bright well-measured stars, we achieved nominal PM precisions of 55 $\mu$as/yr down to 11 $\mu$as/yr . To demonstrate the potential and limitations of our PM data set, we analysed the cluster NGC 1850 and showcase a selection of different science applications. The precision of the PM measurements allows us to disentangle the kinematics of the various stellar populations that are present in the HST field. The cluster has a centre-of-mass motion that is different from the surrounding old field stars and also differs from the mean motion of a close-by group of very young stars. We determined the velocity dispersion of field stars to be 0.128 +/- 0.003 mas/yr (corresponding to 30.3 +/- 0.7 km/s). The velocity dispersion of the cluster inferred from the PM data set most probably overestimates the true value, suggesting that the precision of the measurements at this stage is not sufficient for a reliable analysis of the internal kinematics of extra-galactic star clusters. Finally, we exploit the PM-cleaned catalogue of likely cluster members to determine any radial segregation between fast and slowly-rotating stars, finding that the former are more centrally concentrated. With this paper, we also release the astro-photometric catalogues for each cluster.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.17429-b31b1b.svg)](https://arxiv.org/abs/2406.17429) | **Imaging of I Zw 18 by JWST: II. Spatially resolved star formation history**  |
|| G. Bortolini, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2024-06-26*|
|*Comments*| *14 pages, 11 figures, accepted for publications in Astronomy & Astrophysics (section "4. Extragalactic astronomy")*|
|**Abstract**|            The blue compact dwarf galaxy I Zw 18 is one of the most metal-poor ($Z \sim 3% Z_{\sun}$) star-forming galaxies in the local Universe. Its evolutionary status has sparked debate within the astronomical community. We aim to investigate the stellar populations of I Zw 18 in the near-IR using JWST/NIRCam's high spatial resolution and sensitivity. Additionally, we aim to derive the galaxy's spatially resolved star formation history (SFH) over the last 1 Gyr and provide constraints for older epochs. We used DOLPHOT to measure positions and fluxes of point sources in the F115W and F200W filters' images of I Zw 18. To derive I Zw 18's SFH, we applied the color-magnitude diagram (CMD) fitting technique SFERA 2.0, using two independent sets of stellar models. Our analysis reveals three main stellar populations: one younger than $\sim30$ Myr, mainly in the northwest star-forming (SF) region; an intermediate-age population ($\sim 100 - 800$ Myr) in the southeast SF region; and a red and faint population linked to the underlying halo, older than 1 Gyr and possibly as old as 13.8 Gyr. The main body of the galaxy shows a very low star formation rate (SFR) of $\sim 10^{-4} M_{\odot} \text{yr}^{-1}$ between 1 and 13.8 Gyr ago. In the last billion years, I Zw 18 shows increasing SF, with strong bursts around $\sim10$ and $\sim100$ Myr ago. Component C mirrors the main body's evolution but with lower SFRs. Our findings confirm that I Zw 18 contains stars of all ages, indicating it is not a young galaxy but has an old stellar halo, similar to other BCDs. The low SF activity over the past billion years supports the "slow cooking" dwarf scenario, explaining its low metal content. Currently, the galaxy is undergoing its strongest SF episode ($\sim 0.6 M_{\odot} \text{yr}^{-1}$) mainly in the northwest region, likely due to a recent gravitational interaction with Component C.         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2406.17347/./Figs/pm_correction.png', 'tmp_2406.17347/./Figs/NGC1850_cmd_populations_paper.png', 'tmp_2406.17347/./Figs/ngc1850_vpd_populations_paper.png', 'tmp_2406.17347/./Figs/NGC1850_paper.png']
copying  tmp_2406.17347/./Figs/pm_correction.png to _build/html/
copying  tmp_2406.17347/./Figs/NGC1850_cmd_populations_paper.png to _build/html/
copying  tmp_2406.17347/./Figs/ngc1850_vpd_populations_paper.png to _build/html/
copying  tmp_2406.17347/./Figs/NGC1850_paper.png to _build/html/
exported in  _build/html/2406.17347.md
    + _build/html/tmp_2406.17347/./Figs/pm_correction.png
    + _build/html/tmp_2406.17347/./Figs/NGC1850_cmd_populations_paper.png
    + _build/html/tmp_2406.17347/./Figs/ngc1850_vpd_populations_paper.png
    + _build/html/tmp_2406.17347/./Figs/NGC1850_paper.png
found figures ['tmp_2406.17429/./Images/F200W_cutout_v2.png', 'tmp_2406.17429/./Images/spatial_distributions.png', 'tmp_2406.17429/./Images/sfh_mass_lum_functions_region1.png', 'tmp_2406

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Hubble Space Telescope proper motions of Large Magellanic Cloud star clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.17347-b31b1b.svg)](https://arxiv.org/abs/2406.17347)<mark>Appeared on: 2024-06-26</mark> -  _27 pages, 15 figures. Accepted for publication in A&A. Astro-photometric catalogues are available at this https URL_

</div>
<div id="authors">

F. Niederhofer, et al. -- incl., <mark>M. Häberle</mark>

</div>
<div id="abstract">

**Abstract:** We present proper motion (PM) measurements for a sample of 23 massive star clusters within the Large Magellanic Cloud using multi-epoch data from the _Hubble_ Space Telescope (HST). We combined archival data from the ACS/WFC and WFC3/UVIS instruments with observations from a dedicated HST programme, resulting in time baselines between 4.7 and 18.2 yr available for PM determinations. For bright well-measured stars, we achieved nominal PM precisions of 55 $\mu$ as yr $^{-1}$ down to 11 $\mu$ as yr $^{-1}$ . To demonstrate the potential and limitations of our PM data set, we analysed the cluster NGC 1850 and showcase a selection of different science applications. The precision of the PM measurements allows us to disentangle the kinematics of the various stellar populations that are present in the HST field. The cluster has a centre-of-mass motion that is different from the surrounding old field stars and also differs from the mean motion of a close-by group of very young stars. We determined the velocity dispersion of field stars to be $0.128\pm0.003$ mas yr $^{-1}$ (corresponding to $30.3\pm0.7$ km s $^{-1}$ ). The velocity dispersion of the cluster inferred from the PM data set most probably overestimates the true value, suggesting that the precision of the measurements at this stage is not sufficient for a reliable analysis of the internal kinematics of extra-galactic star clusters.  Finally, we exploit the PM-cleaned catalogue of likely cluster members to determine any radial segregation between fast and slowly-rotating stars, finding that the former are more centrally concentrated.  With this paper, we also release the astro-photometric catalogues for each cluster.

</div>

<div id="div_fig1">

<img src="tmp_2406.17347/./Figs/pm_correction.png" alt="Fig11" width="100%"/>

**Figure 11. -** Outline of the procedure to correct the PMs for high-frequency systematic effects. Panel (a) shows the VPD of sources towards NGC 1850. Likely cluster members were selected based on their PMs (within the red circle, which has a radius of 0.1 mas yr$^{-1}$ in the case of NGC 1850) and their positions in the CMD. Panel (b) shows the $m_{\rm F814W}$ vs $m_{\rm F438W}-m_{\rm F814W}$ CMD of all sources with measured PMs towards NGC 1850 (grey dots) and the final selection of well-measured cluster stars (black dots). Panels (c) and (d) show, separately for each PM component, the maps of the local mean raw PM field (before the high-frequency correction). Each point in the map is colour-coded according to the median PM of its closest 200 well-measured cluster stars. The corresponding maps of the a posteriori corrected PMs are displayed in panels (e) and (f). The points in these maps are coloured according to the same range in colour as in panels (c) and (d). (*fig:PM_correction*)

</div>
<div id="div_fig2">

<img src="tmp_2406.17347/./Figs/NGC1850_cmd_populations_paper.png" alt="Fig15.1" width="50%"/><img src="tmp_2406.17347/./Figs/ngc1850_vpd_populations_paper.png" alt="Fig15.2" width="50%"/>

**Figure 15. -** The different populations in the field of NGC 1850. _Left_: $m_{\rm F814W}$ vs $m_{\rm F336W}-m_{\rm F814W}$ CMD highlighting the selected members of NGC 1850 (orange dots), NGC 1850B (red diamonds) and old field stars (blue squares). _Right_: VPD of the same populations.
 (*fig:ngc1850_populations*)

</div>
<div id="div_fig3">

<img src="tmp_2406.17347/./Figs/NGC1850_paper.png" alt="Fig4" width="100%"/>

**Figure 4. -** Three-colour image of NGC 1850, created from HST observations in the F336W (blue), F438W (green) and F814W (red) filters. Member stars of
NGC 1850 are selected based on their motion and distance
from the cluster centre (white solid circle). Stars belonging to
the young group called NGC 1850B are selected as the young
sequence in the CMD that are located within the white dashed
circle and show consistent motions.  (*fig:ngc1850b*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.17347"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Imaging of I Zw 18 by JWST: II. Spatially resolved star formation history

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.17429-b31b1b.svg)](https://arxiv.org/abs/2406.17429)<mark>Appeared on: 2024-06-26</mark> -  _14 pages, 11 figures, accepted for publications in Astronomy & Astrophysics (section "4. Extragalactic astronomy")_

</div>
<div id="authors">

G. Bortolini, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The blue compact dwarf (BCD) galaxy I Zw 18 is one of the most metal-poor ( $Z \sim 3\%   Z_{\sun}$ ) star-forming galaxies known in the local Universe. Since its discovery, the evolutionary status of this system has been at the center of numerous debates within the astronomical community. We aim to probe and resolve the stellar populations of I Zw 18 in the near-IR using JWST/NIRCam's unprecedented imaging spatial resolution and sensitivity. Additionally, our goal is to derive the spatially resolved star formation history (SFH) of the galaxy within the last $1$ Gyr, and to provide constraints for older epochs. We used the point spread function fitting photometry package \texttt{DOLPHOT} to measure positions and fluxes of point sources in the $F115W$ and $F200W$ filters' images of I Zw 18, acquired as part of the JWST GTO ID 1233 (PI: Meixner). Furthermore, to derive I Zw 18's SFH, we applied a state-of-the-art color-magnitude diagram (CMD) fitting technique ( \texttt{SFERA} 2.0), using two independent sets of stellar models: PARSEC-COLIBRI and MIST. Our analysis of I Zw 18's CMD reveal three main stellar populations: one younger than $\sim30$ Myr, mainly associated with the northwest star-forming (SF) region; an intermediate-age population ( $\sim 100 - 800$ Myr), associated with the southeast SF region; and a red and faint population, linked to the underlying halo of the galaxy, older than $1$ Gyr and possibly as old as $13.8$ Gyr. The main body of the galaxy shows a very low star formation rate (SFR) of $\sim 10^{-4} \; M_{\odot} \; {yr}^{-1}$ between $1$ and $13.8$ Gyr ago. In the last billion years, I Zw 18 shows an increasing trend, culminating in two strong bursts of SF around $\sim 10$ and $\sim100$ Myr ago. Notably, I Zw 18 Component C mimics the evolution of the main body, but with lower SFRs on average. Our results confirm that I Zw 18 is populated by stars of all ages, without any major gaps. Thus, I Zw 18 is not a truly young galaxy, but rather a system characterized by an old underlying stellar halo, in agreement with what has been found in other BCDs by similar studies. The low SF activity exhibited at epochs older than $1$ Gyr is in agreement with the "slow cooking’’ dwarf scenario proposed in the literature, and could have contributed to its low metal content. The galaxy is now experiencing its strongest episode of star formation ( $\sim 0.6   M_{\sun}   yr^{-1}$ ) mainly located in the northwest region. A recent gravitational interaction between the main body and Component C is the most likely explanation for this starburst.

</div>

<div id="div_fig1">

<img src="tmp_2406.17429/./Images/F200W_cutout_v2.png" alt="Fig1" width="100%"/>

**Figure 1. -** $F200W$$(\sim K)$ JWST/NIRCam image cutout displaying I Zw 18's main body (southeast) and Component C (northwest). We divided the main body of the galaxy in four different regions, going from the two inner star-forming regions (labeled R1 and R2, respectively), a middle section (R3), and the outermost region (R4). The Component C is also labeled in the image. In the bottom-left corner is shown a physical scale corresponding to $\sim 5.7$\arcsec$$($\sim 500$ pc). (*F200W_cutout*)

</div>
<div id="div_fig2">

<img src="tmp_2406.17429/./Images/spatial_distributions.png" alt="Fig5" width="100%"/>

**Figure 5. -** F200W image of I Zw 18 with the positions of three bona fide stellar populations being overplotted and selected in the CMD using PARSEC-COLIBRI isochrones. A population of upper MS stars younger than $30$ Myr (blue circles, left panel), AGB stars of intermediate ages between $0.1$ and $1$ Gyr (green circles, middle panel), and a population of RGB and low-mass AGB stars older than $1$ Gyr (red circles, right panel). The insert panels on the bottom-left show the respected position in the CMD of the selected bona fide stellar populations. (*fig:spatial distribution*)

</div>
<div id="div_fig3">

<img src="tmp_2406.17429/./Images/sfh_mass_lum_functions_region1.png" alt="Fig7.1" width="50%"/><img src="tmp_2406.17429/./Images/data_model_residual_region1.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** \texttt{SFERA} 2.0 results for region 1. Panels (a)-(b): recovered SFH and mass using PARSEC-COLIBRI (orange line) and MIST (blue line) isochrones. The shaded areas mark the 16-84 percentiles. Panels (c)-(d): Data (black points) vs. model CMD luminosity functions for different luminosity intervals (indicated on the upper-right). Again, the PARSEC-COLIBRI solution is in orange, while the MIST in blue. Panels (e)-(g): Observational CMD, \texttt{SFERA} 2.0 best fit model CMD and normalized residuals for each CMD cell ($data - model/\sqrt{model}$). The diagrams are color-coded according to the number of stars in each cell (see color-bar in the upper-right corner). The dark-shade masks the region of the CMD that is below our fiducial $50\%$ completeness limit. This region is excluded from the fitting procedure. (*fig:region1_results*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.17429"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

104  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
